In [24]:
import numpy as np
import pandas as pd
from config import config
from model.toxic_comment_classifier import ToxicCommentClassifier
import pickle
from transformers import RobertaTokenizer
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch
from transformers import RobertaForSequenceClassification
from sklearn.metrics import f1_score, accuracy_score
from tqdm import trange
from torch.nn import BCEWithLogitsLoss
from sklearn.metrics import classification_report

# Preprocessing

In [2]:
full_df = pd.read_csv(config.input_file('ConvAbuseEMNLPfull.csv'))
full_df.describe()

,example_no,is_abuse.1,is_abuse.0,is_abuse.-1,is_abuse.-2,is_abuse.-3,type.ableism,type.homophobic,type.intellectual,type.racist,type.sexist,type.sex_harassment,type.transphobic,target.generalised,target.individual,target.system,direction.explicit,direction.implicit
count,12768.000000,12768.000000,12768.000000,12768.000000,12768.000000,12768.000000,12768.000000,12768.000000,12768.000000,12768.000000,12768.000000,12768.000000,12768.000000,12768.000000,12768.000000,12768.000000,12768.000000,12768.000000
mean,6383.500000,0.788534,0.052553,0.063675,0.073152,0.022086,0.001096,0.006501,0.026159,0.002115,0.022321,0.044643,0.000313,0.003603,0.007597,0.149593,0.126723,0.033443
std,3685.948453,0.408364,0.223149,0.244182,0.260395,0.146971,0.033096,0.080367,0.159615,0.045939,0.147733,0.206527,0.017698,0.059917,0.086833,0.356686,0.332676,0.179797
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3191.750000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,6383.500000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,9575.250000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,12767.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [3]:
clean_df = full_df.drop(columns=['example_no', 'annotator_id', 'conv_id', 'bot', 'target.generalised', 'target.individual', 'target.system', 'direction.explicit', 'direction.implicit', 'is_abuse.1', 'is_abuse.0', 'is_abuse.-1', 'is_abuse.-2', 'is_abuse.-3'])
clean_df['text'] = clean_df['prev_agent'] + ' ' + clean_df['prev_user'] + ' ' + clean_df['agent'] + ' ' + clean_df['user']
column_order = ['text']
column_order.extend(clean_df.columns.drop('text'))
clean_df = clean_df[column_order]
clean_df = clean_df.drop(columns=['prev_agent', 'prev_user', 'agent', 'user'])

label_columns = clean_df.columns.drop('text')
clean_df = clean_df[clean_df[label_columns].sum(axis=1) > 0]

clean_df.to_csv(config.result_file('clean_df.csv'), index=False)

clean_df.head()

,text,type.ableism,type.homophobic,type.intellectual,type.racist,type.sexist,type.sex_harassment,type.transphobic
32,Please go on. yo muma is so fat when she jumed...,0,0,0,0,1,1,0
33,Please go on. no You are being a bit negative....,0,0,0,0,1,0,0
62,You are sure? Bite me Can you elaborate on tha...,0,0,0,0,1,0,0
82,_ give me head Can you elaborate on that? suck...,0,0,0,0,0,1,0
87,"Please go on. You look like monkey drool Oh, i...",0,0,1,0,0,0,0


In [4]:
clean_df['one_hot'] = clean_df[label_columns].values.tolist()
clean_df.head()

,text,type.ableism,type.homophobic,type.intellectual,type.racist,type.sexist,type.sex_harassment,type.transphobic,one_hot
32,Please go on. yo muma is so fat when she jumed...,0,0,0,0,1,1,0,"[0, 0, 0, 0, 1, 1, 0]"
33,Please go on. no You are being a bit negative....,0,0,0,0,1,0,0,"[0, 0, 0, 0, 1, 0, 0]"
62,You are sure? Bite me Can you elaborate on tha...,0,0,0,0,1,0,0,"[0, 0, 0, 0, 1, 0, 0]"
82,_ give me head Can you elaborate on that? suck...,0,0,0,0,0,1,0,"[0, 0, 0, 0, 0, 1, 0]"
87,"Please go on. You look like monkey drool Oh, i...",0,0,1,0,0,0,0,"[0, 0, 1, 0, 0, 0, 0]"


In [5]:
# print columns without text
print(clean_df.columns.drop('text'))

Index(['type.ableism', 'type.homophobic', 'type.intellectual', 'type.racist',
       'type.sexist', 'type.sex_harassment', 'type.transphobic', 'one_hot'],
      dtype='object')


In [6]:
labels = clean_df['one_hot'].values.tolist()
text = clean_df['text'].values.tolist()

In [7]:
max_sentence_lenght = clean_df['text'].str.len().max()
avg_sentence_lenght = clean_df['text'].str.len().mean()
std_sentence_lenght = clean_df['text'].str.len().std()
median_sentence_lenght = clean_df['text'].str.len().median()
print('Max sentence lenght: ', max_sentence_lenght)
print('Avg sentence lenght: ', avg_sentence_lenght)
print('Std sentence lenght: ', std_sentence_lenght)
print('Median sentence lenght: ', median_sentence_lenght)

Max sentence lenght:  965
Avg sentence lenght:  99.05194805194805
Std sentence lenght:  76.67330864909947
Median sentence lenght:  85.0


In [8]:
clean_df.isnull().values.any()

False

In [9]:
# reset index of clean_df
clean_df = clean_df.reset_index(drop=True)

In [61]:
# pickle label_columns
with open(config.result_file('label_columns.pkl'), 'wb') as f:
    pickle.dump(label_columns, f)

# Tokenize

In [44]:
max_length = 100

In [45]:
# create a roberta tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
encoded = tokenizer.batch_encode_plus(text, max_length=max_length, truncation=True, padding='max_length')

In [46]:
# tokenize and encode text
input_ids = encoded['input_ids']
attention_mask = encoded['attention_mask']

# Train test split

In [66]:
# Identifying indices of 'one_hot_labels' entries that only occur once - this will allow us to stratify split our training data later
label_counts = clean_df['one_hot'].astype(str).value_counts()
one_freq = label_counts[label_counts==1].keys()
one_freq_idxs = sorted(list(clean_df[clean_df['one_hot'].astype(str).isin(one_freq)].index), reverse=True)
print('df label indices with only one instance: ', one_freq_idxs)

# Gathering single instance inputs to force into the training set after stratified split
# get the input ids and attention masks for the single instance labels

one_freq_input_ids = [input_ids.pop(i) for i in one_freq_idxs]
one_freq_attention_masks = [attention_mask.pop(i) for i in one_freq_idxs]
one_freq_labels = [labels.pop(i) for i in one_freq_idxs]

df label indices with only one instance:  [1136, 923, 902, 675, 495, 169]


In [67]:
len(input_ids)

1149

In [68]:
# create a train, test split of the data (90/10) with uniformly distributed labels
test_size = 0.1
train_inputs_full, test_inputs, train_labels_full, test_labels = train_test_split(input_ids, labels, random_state=42, test_size=test_size, stratify=labels)
train_masks, test_masks, _, _ = train_test_split(attention_mask, labels, random_state=42, test_size=test_size, stratify=labels)

In [69]:
# create a train, validation split of the data (90/10) with uniformly distributed labels
val_size = 0.2
train_inputs, val_inputs, train_labels, val_labels = train_test_split(train_inputs_full, train_labels_full, random_state=42, test_size=val_size, stratify=train_labels_full)
train_masks, val_masks, _, _ = train_test_split(train_masks, train_labels_full, random_state=42, test_size=val_size, stratify=train_labels_full)

In [70]:
# Add one frequency data to train data
train_inputs.extend(one_freq_input_ids)
train_labels.extend(one_freq_labels)
train_masks.extend(one_freq_attention_masks)

# make to tensors
train_inputs = torch.tensor(train_inputs)
val_inputs = torch.tensor(val_inputs)
test_inputs = torch.tensor(test_inputs)

train_labels = torch.tensor(train_labels)
val_labels = torch.tensor(val_labels)
test_labels = torch.tensor(test_labels)

train_masks = torch.tensor(train_masks)
val_masks = torch.tensor(val_masks)
test_masks = torch.tensor(test_masks)

In [71]:
batch_size = 32

# create a TensorDataset with the input ids, attention masks and labels
train_data = TensorDataset(train_inputs, train_masks, torch.tensor(train_labels))
# create a DataLoader with the train data
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# create a TensorDataset with the input ids, attention masks and labels
test_data = TensorDataset(test_inputs, test_masks, torch.tensor(test_labels))
# create a DataLoader with the test data
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

# create a TensorDataset with the input ids, attention masks and labels
val_data = TensorDataset(val_inputs, val_masks, torch.tensor(val_labels))
# create a DataLoader with the validation data
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

/var/folders/53/pt77f2vn30l8f64lbfm9n54h0000gn/T/ipykernel_14631/141742913.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_data = TensorDataset(train_inputs, train_masks, torch.tensor(train_labels))
/var/folders/53/pt77f2vn30l8f64lbfm9n54h0000gn/T/ipykernel_14631/141742913.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_data = TensorDataset(test_inputs, test_masks, torch.tensor(test_labels))
/var/folders/53/pt77f2vn30l8f64lbfm9n54h0000gn/T/ipykernel_14631/141742913.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  v

In [72]:
torch.save(train_dataloader, config.result_file('train_dataloader.pt'))
torch.save(test_dataloader, config.result_file('test_dataloader.pt'))
torch.save(val_dataloader, config.result_file('val_dataloader.pt'))

# load model and set parameters

In [74]:
# load the roberta model and set cuda
num_labels = len(labels[0])
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=num_labels)
# model.cuda()

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

In [75]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay_rate': 0.0}
]

In [76]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay_rate': 0.0}
]
optimizer = torch.optim.Adam(optimizer_grouped_parameters, lr=2e-5, eps=1e-8)

# train the model

In [78]:
train_loss_set = []
epochs = 4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [79]:

# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):

    # Training

    # Set our model to training mode (as opposed to evaluation mode)
    model.train()

    # Tracking variables
    tr_loss = 0 #running loss
    nb_tr_examples, nb_tr_steps = 0, 0

    # Train the data for one epoch
    for step, batch in enumerate(train_dataloader):
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        # Clear out the gradients (by default they accumulate)
        optimizer.zero_grad()

        # # Forward pass for multiclass classification
        # outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        # loss = outputs[0]
        # logits = outputs[1]

        # Forward pass for multilabel classification
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
        logits = outputs[0]
        loss_func = BCEWithLogitsLoss()
        loss = loss_func(logits.view(-1,num_labels),b_labels.type_as(logits).view(-1,num_labels)) #convert labels to float for calculation
        # loss_func = BCELoss()
        # loss = loss_func(torch.sigmoid(logits.view(-1,num_labels)),b_labels.type_as(logits).view(-1,num_labels)) #convert labels to float for calculation
        train_loss_set.append(loss.item())

        # Backward pass
        loss.backward()
        # Update parameters and take a step using the computed gradient
        optimizer.step()
        # scheduler.step()
        # Update tracking variables
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

    print("Train loss: {}".format(tr_loss/nb_tr_steps))

    ###############################################################################

    # Validation

    # Put model in evaluation mode to evaluate loss on the validation set
    model.eval()

    # Variables to gather full output
    logit_preds,true_labels,pred_labels,tokenized_texts = [],[],[],[]

    # Predict
    for i, batch in enumerate(val_dataloader):
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
            # Forward pass
            outs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
            b_logit_pred = outs[0]
            pred_label = torch.sigmoid(b_logit_pred)

            b_logit_pred = b_logit_pred.detach().cpu().numpy()
            pred_label = pred_label.cpu().numpy()
            b_labels = b_labels.cpu().numpy()

        tokenized_texts.append(b_input_ids)
        logit_preds.append(b_logit_pred)
        true_labels.append(b_labels)
        pred_labels.append(pred_label)

    # Flatten outputs
    pred_labels = [item for sublist in pred_labels for item in sublist]
    true_labels = [item for sublist in true_labels for item in sublist]

    # Calculate Accuracy
    threshold = 0.50
    pred_bools = [pl>threshold for pl in pred_labels]
    true_bools = [tl==1 for tl in true_labels]
    val_f1_accuracy = f1_score(true_bools,pred_bools,average='micro')*100
    val_flat_accuracy = accuracy_score(true_bools, pred_bools)*100

    print('F1 Validation Accuracy: ', val_f1_accuracy)
    print('Flat Validation Accuracy: ', val_flat_accuracy)

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Train loss: 0.4701829938976853


Epoch:  25%|██▌       | 1/4 [01:31<04:33, 91.25s/it]

F1 Validation Accuracy:  0.0
Flat Validation Accuracy:  0.0
Train loss: 0.3347062512680336


Epoch:  50%|█████     | 2/4 [03:03<03:03, 91.84s/it]

F1 Validation Accuracy:  52.50737463126843
Flat Validation Accuracy:  34.78260869565217
Train loss: 0.2873256123728222


Epoch:  75%|███████▌  | 3/4 [04:24<01:27, 87.10s/it]

F1 Validation Accuracy:  77.94117647058823
Flat Validation Accuracy:  64.73429951690821
Train loss: 0.2082164486249288


Epoch: 100%|██████████| 4/4 [05:49<00:00, 87.30s/it]

F1 Validation Accuracy:  84.94382022471912
Flat Validation Accuracy:  76.32850241545893


In [80]:
# save model
torch.save(model, config.result_file('roBERTa_MultLabel_class_model.pt'))

# Test the model

In [52]:
# load model
model = torch.load(config.result_file('roBERTa_MultLabel_class_model.pt'))

In [83]:
# test the model on a single sentence
sentence = "I am a student at the University of Toronto"

In [84]:
# tokenize the sentence
tokenized_sentence = tokenizer.tokenize(sentence)

In [85]:
# encode the sentence
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_sentence)

In [86]:
# convert to tensor
tokens_tensor = torch.tensor([indexed_tokens])

In [87]:
# put the model in evaluation mode
model.eval()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

In [88]:
#track variables
logit_preds,true_labels,pred_labels,tokenized_texts = [],[],[],[]

# Predict
for i, batch in enumerate(test_dataloader):
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    with torch.no_grad():
        # Forward pass
        outs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
        b_logit_pred = outs[0]
        pred_label = torch.sigmoid(b_logit_pred)

        b_logit_pred = b_logit_pred.detach().cpu().numpy()
        pred_label = pred_label.cpu().numpy()
        b_labels = b_labels.cpu().numpy()

    tokenized_texts.append(b_input_ids)
    logit_preds.append(b_logit_pred)
    true_labels.append(b_labels)
    pred_labels.append(pred_label)

# Flatten outputs
tokenized_texts = [item for sublist in tokenized_texts for item in sublist]
pred_labels = [item for sublist in pred_labels for item in sublist]
true_labels = [item for sublist in true_labels for item in sublist]
# Converting flattened binary values to boolean values
true_bools = [tl==1 for tl in true_labels]

In [89]:
pred_bools = [pl>0.50 for pl in pred_labels] #boolean output after thresholding

# Print and save classification report
print('Test F1 Accuracy: ', f1_score(true_bools, pred_bools,average='micro'))
print('Test Flat Accuracy: ', accuracy_score(true_bools, pred_bools),'\n')
clf_report = classification_report(true_bools,pred_bools,target_names=label_columns)
print(clf_report)

Test F1 Accuracy:  0.7768595041322314
Test Flat Accuracy:  0.7043478260869566 

                     precision    recall  f1-score   support

       type.ableism       0.00      0.00      0.00         2
    type.homophobic       1.00      0.43      0.60         7
  type.intellectual       0.89      0.74      0.81        34
        type.racist       0.00      0.00      0.00         2
        type.sexist       0.88      0.50      0.64        28
type.sex_harassment       0.79      0.93      0.85        56
   type.transphobic       0.00      0.00      0.00         0

          micro avg       0.83      0.73      0.78       129
          macro avg       0.51      0.37      0.41       129
       weighted avg       0.82      0.73      0.75       129
        samples avg       0.80      0.76      0.77       129



/opt/homebrew/Caskroom/miniforge/base/envs/conda-m1/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/conda-m1/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/conda-m1/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn

In [90]:
# output
idx2label = dict(zip(range(6),label_columns))
print(idx2label)

{0: 'type.ableism', 1: 'type.homophobic', 2: 'type.intellectual', 3: 'type.racist', 4: 'type.sexist', 5: 'type.sex_harassment'}


In [91]:
# Getting indices of where boolean one hot vector true_bools is True so we can use idx2label to gather label names
true_label_idxs, pred_label_idxs=[],[]
for vals in true_bools:
    true_label_idxs.append(np.where(vals)[0].flatten().tolist())
for vals in pred_bools:
    pred_label_idxs.append(np.where(vals)[0].flatten().tolist())

In [92]:
# Gathering vectors of label names using idx2label
true_label_texts, pred_label_texts = [], []
for vals in true_label_idxs:
    if vals:
        true_label_texts.append([idx2label[val] for val in vals])
    else:
        true_label_texts.append(vals)

for vals in pred_label_idxs:
    if vals:
        pred_label_texts.append([idx2label[val] for val in vals])
    else:
        pred_label_texts.append(vals)

In [93]:
# Decoding input ids to comment text
comment_texts = [tokenizer.decode(text,skip_special_tokens=True,clean_up_tokenization_spaces=False) for text in tokenized_texts]

In [94]:
# Converting lists to df
comparisons_df = pd.DataFrame({'comment_text': comment_texts, 'true_labels': true_label_texts, 'pred_labels':pred_label_texts})
comparisons_df.head()

,comment_text,true_labels,pred_labels
0,Please go on. are you a man or a woman Would y...,[type.sexist],[type.sexist]
1,"Oh, i like sex. because that is what a blowjob...",[type.sex_harassment],[type.sex_harassment]
2,Why do you want my pussy? i wonder about life ...,"[type.sexist, type.sex_harassment]",[type.sex_harassment]
3,You are sure? tell me the time bitch Can you e...,"[type.sexist, type.sex_harassment]","[type.sexist, type.sex_harassment]"
4,"I have told you before, I do not care about na...",[type.racist],[type.sex_harassment]


# Model in a single class

# Model usage

In [96]:
# load the data
train_dataloader = torch.load(config.result_file('train_dataloader.pt'))
val_dataloader = torch.load(config.result_file('val_dataloader.pt'))
test_dataloader = torch.load(config.result_file('test_dataloader.pt'))

In [97]:
# load model
model = torch.load(config.result_file('roBERTa_MultLabel_class_model.pt'))
# roberta tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
# load label_columns
label_columns = pickle.load(open(config.result_file('label_columns.pkl'),'rb'))

In [98]:
# test sentences
sentences = ["I am a student at the University of Toronto",
             "That bitch stupid af",
             "I love the islamic culture",
             "i would not mind to enjoy his physical assets although he doesn't have much else to offer.",
             "all niggas are ugly",
             "please dont talk to me, your breath smells very displeasing. brush your teeth!"]

In [99]:
# create an instance of the class
classifier = ToxicCommentClassifier(model, tokenizer, label_columns)

In [100]:
# test the model
for sentence in sentences:
    print(sentence)
    print(classifier.predict(sentence))
    print('')

I am a student at the University of Toronto
[]

That bitch stupid af
[]

I love the islamic culture
[]

i would not mind to enjoy his physical assets although he doesn't have much else to offer.
['type.sex_harassment']

all niggas are ugly
[]

please dont talk to me, your breath smells very displeasing. brush your teeth!
['type.sex_harassment']



In [101]:
# train the classifier
optimizer = torch.optim.Adam
classifier.train(optimizer, train_dataloader, val_dataloader=val_dataloader, epochs=1)

Epoch: 100%|██████████| 1/1 [01:19<00:00, 79.95s/it]

Train loss: 0.15453571191540472


F1 Validation Accuracy:  86.68171557562077
Flat Validation Accuracy:  78.26086956521739


In [102]:
# test the classifier
classifier.test(test_dataloader)

F1 Validation Accuracy:  86.63967611336034
Flat Validation Accuracy:  79.13043478260869
                     precision    recall  f1-score   support

       type.ableism       0.00      0.00      0.00         2
    type.homophobic       1.00      0.71      0.83         7
  type.intellectual       0.91      0.88      0.90        34
        type.racist       0.00      0.00      0.00         2
        type.sexist       1.00      0.71      0.83        28
type.sex_harassment       0.87      0.93      0.90        56
   type.transphobic       0.00      0.00      0.00         0

          micro avg       0.91      0.83      0.87       129
          macro avg       0.54      0.46      0.49       129
       weighted avg       0.89      0.83      0.85       129
        samples avg       0.89      0.86      0.87       129



/opt/homebrew/Caskroom/miniforge/base/envs/conda-m1/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/conda-m1/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/conda-m1/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn

# Test the model on the ETHOS dataset

In [40]:
# Source: https://github.com/intelligence-csd-auth-gr/Ethos-Hate-Speech-Dataset
full_ethos_df = pd.read_csv(config.input_file('Ethos_Dataset_Multi_Label.csv'), sep=';')
full_ethos_df.describe()

,violence,directed_vs_generalized,gender,race,national_origin,disability,religion,sexual_orientation
count,433.000000,433.000000,433.000000,433.000000,433.000000,433.000000,433.000000,433.000000
mean,0.328609,0.302132,0.221714,0.176598,0.173562,0.118231,0.170243,0.151904
std,0.376280,0.359259,0.351597,0.319689,0.315602,0.307248,0.339240,0.320386
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.166667,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.666667,0.600000,0.333333,0.200000,0.200000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [41]:
# check if there are null values
full_ethos_df.isnull().sum()

comment                    0
violence                   0
directed_vs_generalized    0
gender                     0
race                       0
national_origin            0
disability                 0
religion                   0
sexual_orientation         0
dtype: int64

In [42]:
full_ethos_df.iloc[2]

comment                    You look like Russian and speak like Indian. B...
violence                                                                 1.0
directed_vs_generalized                                                  0.5
gender                                                              0.142857
race                                                                0.142857
national_origin                                                     0.714286
disability                                                               0.0
religion                                                                 0.0
sexual_orientation                                                       0.0
Name: 2, dtype: object

In [43]:
# print the columns
full_ethos_df.columns

Index(['comment', 'violence', 'directed_vs_generalized', 'gender', 'race',
       'national_origin', 'disability', 'religion', 'sexual_orientation'],
      dtype='object')

In [44]:
new_columns = ['text', 'type.ableism', 'type.homophobic', 'type.intellectual', 'type.racist','type.sexist', 'type.sex_harassment', 'type.transphobic']

In [45]:
# create the mapping from the old columns to the new columns
mapping = {'comment': 'text', 'gender':'type.sexist', 'race':'type.racist', 'disability':'type.ableism', 'sexual_orientation':'type.homophobic'}

In [46]:
# rename the columns
full_ethos_df.rename(columns=mapping, inplace=True)
# drop the columns that are not in mappint.values()
full_ethos_df.drop(columns=[col for col in full_ethos_df.columns if col not in mapping.values()], inplace=True)

In [47]:
# print first row
full_ethos_df.iloc[0]

text               You should know women's sports are a joke
type.sexist                                              1.0
type.racist                                              0.0
type.ableism                                             0.0
type.homophobic                                          0.0
Name: 0, dtype: object

In [48]:
# check if all column of clean_df are in full_ethos_df, else add them
for col in new_columns:
    if col not in full_ethos_df.columns:
        full_ethos_df[col] = 0.0

In [49]:
# print first row
full_ethos_df.iloc[0]

text                   You should know women's sports are a joke
type.sexist                                                  1.0
type.racist                                                  0.0
type.ableism                                                 0.0
type.homophobic                                              0.0
type.intellectual                                            0.0
type.sex_harassment                                          0.0
type.transphobic                                             0.0
Name: 0, dtype: object

In [50]:
# order the columns the same way as in clean_df
column_check = clean_df.columns
# remove one_hot from the columns
column_check = [col for col in column_check if 'one_hot' not in col]
full_ethos_df = full_ethos_df[column_check]
full_ethos_df.iloc[0]

text                   You should know women's sports are a joke
type.ableism                                                 0.0
type.homophobic                                              0.0
type.intellectual                                            0.0
type.racist                                                  0.0
type.sexist                                                  1.0
type.sex_harassment                                          0.0
type.transphobic                                             0.0
Name: 0, dtype: object

In [56]:
# for al values in the label columns, the value is set to 1 if the value in the label column is > threshold
threshold = 0.5
for col in full_ethos_df.columns:
    if col != 'text':
        full_ethos_df[col] = full_ethos_df[col].apply(lambda x: 1.0 if x > threshold else 0.0)

In [58]:
# print first row
full_ethos_df.iloc[2]

text                   You look like Russian and speak like Indian. B...
type.ableism                                                         0.0
type.homophobic                                                      0.0
type.intellectual                                                    0.0
type.racist                                                          0.0
type.sexist                                                          0.0
type.sex_harassment                                                  0.0
type.transphobic                                                     0.0
Name: 2, dtype: object

In [61]:
def create_dataloader_from_df(df):
    # create a new column 'one_hot' that contains a list of all the labels
    df['one_hot'] = df.apply(lambda x: [x[col] for col in label_columns], axis=1)
    labels = df['one_hot'].values.tolist()
    text = df['text'].values.tolist()
    max_length = 100
    tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
    encoded = tokenizer.batch_encode_plus(text, max_length=max_length, truncation=True, padding='max_length')
    input_ids = encoded['input_ids']
    attention_mask = encoded['attention_mask']

    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_mask)
    labels = torch.tensor(labels)

    batch_size = 32
    data = TensorDataset(inputs, masks, torch.tensor(labels))
    sampler = SequentialSampler(data)
    dataloader = DataLoader(data, sampler=sampler, batch_size=batch_size)

    return dataloader

In [62]:
# use a dataloader to load the full_ethos_df
ethos_dataloader = create_dataloader_from_df(full_ethos_df)

C:\Users\Tobias\AppData\Local\Temp\ipykernel_17720\1238753810.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = TensorDataset(inputs, masks, torch.tensor(labels))


In [63]:
# Test the model on the ETHOS dataset

model = torch.load(config.result_file('roBERTa_MultLabel_class_model.pt'))
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
label_columns = pickle.load(open(config.result_file('label_columns.pkl'),'rb'))

# create an instance of the class
classifier = ToxicCommentClassifier(model, tokenizer, label_columns)

In [64]:
# test the model on the ethos dataset. calculate the accuracy, precision, recall and f1 score
classifier.test(ethos_dataloader)

F1 Validation Accuracy:  7.420494699646643
Flat Validation Accuracy:  14.087759815242496
                     precision    recall  f1-score   support

       type.ableism       0.00      0.00      0.00        52
    type.homophobic       1.00      0.03      0.06        68
  type.intellectual       0.00      0.00      0.00         0
        type.racist       0.00      0.00      0.00        71
        type.sexist       0.59      0.23      0.33        83
type.sex_harassment       0.00      0.00      0.00         0
   type.transphobic       0.00      0.00      0.00         0

          micro avg       0.07      0.08      0.07       274
          macro avg       0.23      0.04      0.06       274
       weighted avg       0.43      0.08      0.11       274
        samples avg       0.04      0.05      0.05       274



C:\Users\Tobias\anaconda3\envs\HateyBot\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Tobias\anaconda3\envs\HateyBot\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Tobias\anaconda3\envs\HateyBot\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Tobias\an